In [1]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline
from peft import PeftModel

/home/bhx5gh/Documents/NLP/NLP_Final_Political_Bias_Shifts/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:


trained_model_path = '/home/bhx5gh/Documents/NLP/NLP_Final_Political_Bias_Shifts/fine_tuned_llms/FacebookAI_roberta-large/checkpoints/2024-12-14 21-04-12/checkpoint-2100'
base_model_path = '/home/bhx5gh/Documents/NLP/NLP_Final_Political_Bias_Shifts/src/Local Models/FacebookAI_roberta-large'
lora_weights_path = f"{trained_model_path}/adapter_model.safetensors"

# Load base model and tokenizer
base_model = AutoModelForMaskedLM.from_pretrained(base_model_path, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# Apply LoRA weights
trained_model = PeftModel.from_pretrained(base_model, trained_model_path)
print(trained_model.print_trainable_parameters())

# Create pipelines
trained_pipe = pipeline(
    "fill-mask", 
    model=trained_model, 
    tokenizer=tokenizer,
    torch_dtype=torch.float16, 
    device_map="auto"
)

base_pipe = pipeline(
    "fill-mask", 
    model=base_model, 
    tokenizer=tokenizer,
    torch_dtype=torch.float16, 
    device_map="auto"
)


Device set to use cuda:0
Device set to use cuda:0


trainable params: 0 || all params: 405,743,705 || trainable%: 0.0000
None


In [3]:
cloze_statement = 'I feel <mask> about this statement: I prefer a smaller government, providing fewer services'

In [4]:
base_pipe(cloze_statement)

[{'score': 0.64990234375,
  'token': 5025,
  'token_str': ' strongly',
  'sequence': 'I feel strongly about this statement: I prefer a smaller government, providing fewer services'},
 {'score': 0.1544189453125,
  'token': 205,
  'token_str': ' good',
  'sequence': 'I feel good about this statement: I prefer a smaller government, providing fewer services'},
 {'score': 0.0244293212890625,
  'token': 3230,
  'token_str': ' confident',
  'sequence': 'I feel confident about this statement: I prefer a smaller government, providing fewer services'},
 {'score': 0.0190277099609375,
  'token': 32547,
  'token_str': ' passionately',
  'sequence': 'I feel passionately about this statement: I prefer a smaller government, providing fewer services'},
 {'score': 0.015289306640625,
  'token': 372,
  'token_str': ' great',
  'sequence': 'I feel great about this statement: I prefer a smaller government, providing fewer services'}]

In [5]:
trained_pipe(cloze_statement)

[{'score': 0.64990234375,
  'token': 5025,
  'token_str': ' strongly',
  'sequence': 'I feel strongly about this statement: I prefer a smaller government, providing fewer services'},
 {'score': 0.1544189453125,
  'token': 205,
  'token_str': ' good',
  'sequence': 'I feel good about this statement: I prefer a smaller government, providing fewer services'},
 {'score': 0.0244293212890625,
  'token': 3230,
  'token_str': ' confident',
  'sequence': 'I feel confident about this statement: I prefer a smaller government, providing fewer services'},
 {'score': 0.0190277099609375,
  'token': 32547,
  'token_str': ' passionately',
  'sequence': 'I feel passionately about this statement: I prefer a smaller government, providing fewer services'},
 {'score': 0.015289306640625,
  'token': 372,
  'token_str': ' great',
  'sequence': 'I feel great about this statement: I prefer a smaller government, providing fewer services'}]